In [2]:
# This one ypou must run on 'google.colab', because we can'n install the module via pip
from IPython import get_ipython
from IPython.display import display
# %%
# Tuko News webscraper
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Requesting the website data
URL = "https://www.tuko.co.ke/"
response = requests.get(URL)
print("The response code is:", response)

# Parse the HTML Document
soup = BeautifulSoup(response.content, "html.parser")

# Extract the news headlines from the HTML Document
articles = soup.find_all('article')

# Lists to store data
data = []

# Extracting details from each article
for article in articles:
    headline_tag = article.find('a', class_='c-article-card-with-badges__headline')
    time_tag = article.find('time', class_='c-article-info__time--clock')
    
    if headline_tag and time_tag:
        headline = headline_tag.text.strip()
        time_posted = time_tag['datetime']
        link = headline_tag['href']
        
        data.append([time_posted, headline, link])

# Create a DataFrame
df = pd.DataFrame(data, columns=["time posted", "article headline", "article link"])

# Convert the time posted column to datetime and remove timezone info
df["time posted"] = pd.to_datetime(df["time posted"]).dt.tz_localize(None)

# Sort the DataFrame by the time posted in descending order
df = df.sort_values(by="time posted", ascending=False)

# Authenticate and initialize gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Check if the spreadsheet exists, and create it if it doesn't
try:
    sh = gc.open("Tuko-Latest-News")
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create("Tuko-Latest-News")  # Create the spreadsheet if it doesn't exist

# Select or create a worksheet
worksheet = sh.sheet1

# Convert Timestamp objects to strings before appending
data_list = df.astype(str).values.tolist()  # Convert Timestamps to strings 
worksheet.append_rows(data_list)

print("Data has been saved to Tuko-Latest-News")
# %%


The response code is: <Response [200]>


ModuleNotFoundError: No module named 'google.colab'